### Breast Cancer CNN

In [12]:
from sklearn.datasets import load_breast_cancer
import numpy as np

breast_cancer = load_breast_cancer()
breast_cancer.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [13]:
breast_cancer['data'].shape

(569, 30)

In [14]:
breast_cancer['feature_names']

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [15]:
breast_cancer['target_names']

array(['malignant', 'benign'], dtype='<U9')

### Use 30% of the data as labeled data

In [16]:
data, labels = breast_cancer.data[:,:], breast_cancer.target[:]

num_samples = len(labels)
shuffle_order = np.random.permutation(num_samples)

data = data[shuffle_order, :]
labels = labels[shuffle_order]

data = data / np.amax(data, axis = 0)
labels = labels / np.amax(labels, axis = 0)
data = data.astype('float32')
labels = labels.astype('float32')

print(data.shape)

(569, 30)


In [17]:
X, y = data, labels

train_split = 0.6
n_train = int(train_split * num_samples)
x_train, y_train = X[:n_train], y[:n_train]
x_test, y_test = X[n_train:], y[n_train:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(341, 30)
(341,)
(228, 30)
(228,)


In [18]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['KERAS_BACKEND'] = 'tensorflow'

import matplotlib.pyplot as plt
import numpy as np
import random

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.layers import Activation

In [19]:
num_classes = 2

#reshape data
x_train = x_train.reshape((341,30,1)) / 255.
x_test = x_test.reshape((228,30,1)) / 255.

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('%d train samples, %d test samples'%(x_train.shape[0], x_test.shape[0]))
print('training data shape: ', x_train.shape, y_train.shape)
print('test data shape: ', x_test.shape, y_test.shape)

341 train samples, 228 test samples
training data shape:  (341, 30, 1) (341, 2)
test data shape:  (228, 30, 1) (228, 2)


In [20]:
x_train.shape[1:]

(30, 1)

In [27]:
model = Sequential()

model.add(Conv1D(64, 3, input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling1D(2))

model.add(Conv1D(32, 3))
model.add(Activation('relu'))
model.add(MaxPooling1D(2))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 28, 64)            256       
_________________________________________________________________
activation_9 (Activation)    (None, 28, 64)            0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 14, 64)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 12, 32)            6176      
_________________________________________________________________
activation_10 (Activation)   (None, 12, 32)            0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 192)               0         
__________

In [28]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy'])

In [29]:
model.fit(x_train, y_train, 
          batch_size=16, 
          epochs=50, 
          validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, batch_size=16)

Train on 341 samples, validate on 228 samples
Epoch 1/50
341/341 [==============================] - 0s 1ms/step - loss: 0.6876 - acc: 0.6100 - val_loss: 0.6815 - val_acc: 0.6272
Epoch 2/50
341/341 [==============================] - 0s 448us/step - loss: 0.6782 - acc: 0.6276 - val_loss: 0.6753 - val_acc: 0.6272
Epoch 3/50
341/341 [==============================] - 0s 428us/step - loss: 0.6730 - acc: 0.6276 - val_loss: 0.6696 - val_acc: 0.6272
Epoch 4/50
341/341 [==============================] - 0s 460us/step - loss: 0.6684 - acc: 0.6276 - val_loss: 0.6663 - val_acc: 0.6272
Epoch 5/50
341/341 [==============================] - 0s 457us/step - loss: 0.6653 - acc: 0.6276 - val_loss: 0.6646 - val_acc: 0.6272
Epoch 6/50
341/341 [==============================] - 0s 469us/step - loss: 0.6641 - acc: 0.6276 - val_loss: 0.6630 - val_acc: 0.6272
Epoch 7/50
341/341 [==============================] - 0s 457us/step - loss: 0.6625 - acc: 0.6276 - val_loss: 0.6619 - val_acc: 0.6272
Epoch 8/50
341/341